<a href="https://colab.research.google.com/github/FatemeZamanian/DeepLearning/blob/main/SheikhDetectionBot/sheikhBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyTelegramBotAPI

     |████████████████████████████████| 101 kB 3.0 MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.0.1-py3-none-any.whl size=79552 sha256=a7a2758f850087e9046fd6535ae8b2f9f121497d64a5a67c49a055a6e6ea8d16
  Stored in directory: /root/.cache/pip/wheels/2f/c4/5a/7440a72b0ee7aad2d7ba7b2b44a93953183a30b017f604bad3
Successfully built pyTelegramBotAPI


In [4]:
from tensorflow.keras.models import load_model
import telebot
from telebot import types
import numpy as np
import cv2

In [ ]:
model=load_model('/content/drive/MyDrive/sheikhs.h5')
bot=telebot.TeleBot("Token")
# btns=telebot.types.ReplyKeyboardMarkup(row_width=1)
# btn1=telebot.types.KeyboardButton('Start')
# btns.add(btn1)
@bot.message_handler(commands=['start'])
def wlc(message):
    bot.send_message(message.chat.id,'Hi 👋🏻 please send me a picture👀',
    #reply_markup=btns
     )

@bot.message_handler(content_types=['photo'])
def photo(message):
    fileID = message.photo[-1].file_id
    file_info = bot.get_file(fileID)
    src=file_info.file_path
    downloaded_file = bot.download_file(src)
    with open(f"/content/drive/MyDrive/{src}", 'wb') as new_file:
        new_file.write(downloaded_file)

    img_org = cv2.imread(f"/content/drive/MyDrive/{src}")
    img_RGB = cv2.cvtColor(img_org, cv2.COLOR_BGR2RGB)
    img_resize = cv2.resize(img_RGB, (224, 224))
    
    img_numpy = np.array(img_resize)
    img = img_numpy / 255.0
    final = img.reshape(1, 224, 224, 3)

    y_pred = np.argmax(model.predict(final))

    if y_pred == 0:
        bot.reply_to(message,'normal person')
      
    else:
        bot.reply_to(message,'sheikh')
      

bot.polling()